<h1>Import Packages</h1>

In [1]:
import os
import pandas as pd

<h1>Combine EPC Files into 1 Dataframe</h1>

In [2]:
main_directory = r"C:\Users\Elisha.Zissman\OneDrive - Social Investment Business\Desktop\epc_docs"

In [3]:
#function to extract

def extract_certificates_to_dataframe(folder_path): 

    certificates_file = os.path.join(folder_path, 'certificates.csv') 

    if os.path.exists(certificates_file): 

        df = pd.read_csv(certificates_file)  # Assuming certificates file is CSV, adjust accordingly 

        return df 

    return None 

In [4]:
dataframes = [] 

for folder_name in os.listdir(main_directory): 

    folder_path = os.path.join(main_directory, folder_name) 

    if os.path.isdir(folder_path): 

        df = extract_certificates_to_dataframe(folder_path) 

        if df is not None: 

            dataframes.append(df) 

C:\Users\Elisha.Zissman\AppData\Local\Temp\ipykernel_18156\646180137.py:9: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(certificates_file)  # Assuming certificates file is CSV, adjust accordingly


In [5]:
combined_dataframe = pd.concat(dataframes, ignore_index=True) 

In [6]:
epc = combined_dataframe

<h2>Checkpoint - Initial Download</h2>

In [7]:
# Calculate the number of rows
num_rows = len(epc)

epc['INSPECTION_DATE'] = pd.to_datetime(epc['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = epc['INSPECTION_DATE'].min()
newest_date = epc['INSPECTION_DATE'].max()

# Create the message
message1 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'

In [8]:
print(message1)

contains 1308301 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Preparing Geography Information</h1>

In [9]:
nspl = pd.read_csv('nspl_21.csv')
rural_urban = pd.read_csv('rural_urban.csv')
imd_dec = pd.read_excel('IMD 2019.xlsx', sheet_name=1)
conv = pd.read_csv('lsoa11 to lsoa21.csv')

C:\Users\Elisha.Zissman\AppData\Local\Temp\ipykernel_18156\2495203614.py:1: DtypeWarning: Columns (26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  nspl = pd.read_csv('nspl_21.csv')


In [10]:
conv = conv.rename(columns={'LSOA11CD':'lsoa11', 'LSOA21CD':'lsoa21'})
conv = conv.drop(columns=['ObjectId',
 'LSOA11NM',
 'LSOA21NM',
 'LAD22CD',
 'LAD22NM',
 'LAD22NMW'])

In [11]:
epc = epc.rename(columns={'POSTCODE':'pcds'})

In [12]:
nspl = nspl.drop(columns=['pcd',
 'pcd2',
 'dointr',
 'doterm',
 'usertype',
 'oseast1m',
 'osnrth1m',
 'osgrdind',
 'oa21',
 'cty',
 'ced',
 'laua',
 'ward',
 'nhser',
 'ctry',
 'pcon',
 'ttwa',
 'itl',
 'npark',
 'msoa21',
 'wz11',
 'sicbl',
 'bua22',
 'ru11ind',
 'oac11',
 'lat',
 'long',
 'lep1',
 'lep2',
 'pfa',
 'icb', 'imd'])

In [13]:
rural_urban = rural_urban.rename(columns={'LSOA11CD':'lsoa11'})
rural_urban = rural_urban.drop(columns=['FID', 'LSOA11NM'])

In [14]:
imd_dec = imd_dec.rename(columns={'LSOA code (2011)':'lsoa11'})

<h1>Merging Dataframes</h1>

In [15]:
# merge with postcode lookup

merged_df = epc.merge(nspl, on='pcds', how='left')

<h2>Checkpoint - NSPL Merge</h2>

In [16]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message2 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message2)

contains 1308301 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Removing non England</h1>

In [17]:
# remove non-england regions

merged_df = merged_df[merged_df['rgn'].notna() & merged_df['rgn'].str.startswith('E', na=False)]

<h2>Checkpoint - Removing Non-England</h2>

In [18]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message3 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message3)

contains 1246391 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Converter</h1>

In [19]:
merged_df = merged_df.merge(conv, on='lsoa21')

<h2>Checkpoint - Merging with Converter file</h2>

In [20]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message4 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message4)

contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Urban Rural</h1>

In [21]:
merged_df = merged_df.merge(rural_urban, on='lsoa11', how='left')

<h2>Checkpoint - Converter</h2>

In [22]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message5 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message5)

contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Merge IMD</h1>

In [23]:
merged_df = merged_df.merge(imd_dec, on='lsoa11', how='left')

<h2>Checkpoint - IMD</h2>

In [24]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message6 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message6)

contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Filter by Property Type</h1>

In [25]:
# filter date and type

property_types = ['D1 Non-residential Institutions - Community/Day Centre', 'Community/day centre', 'Non-residential Institutions: Community/Day Centre']
merged_df = merged_df[merged_df['PROPERTY_TYPE'].isin(property_types)].copy()  # Make a copy of the filtered DataFrame

<h2>Checkpoint - Property Type</h2>

In [26]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message7 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message7)

contains 13415 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Filtering most recent</h1>

In [27]:
# Convert 'INSPECTION_DATE' column to datetime
merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])
 
# Sort the DataFrame by 'INSPECTION_DATE' in descending order
merged_df = merged_df.sort_values(by='INSPECTION_DATE', ascending=False)

# Drop duplicates in the 'BUILDING_REFERENCE_NUMBER' column, keeping only the first occurrence (most recent date)
merged_df = merged_df.drop_duplicates(subset='BUILDING_REFERENCE_NUMBER', keep='first')

<h2>Checkpoint - most recent filter</h2>

In [28]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message8 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message8)

contains 12985 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-09-30 00:00:00


<h1>Filter Time Cut off</h1>

In [29]:
# Filter out rows with 'INSPECTION_DATE' after June 30, 2023
cutoff_date = pd.to_datetime('2023-06-30')
merged_df = merged_df[merged_df['INSPECTION_DATE'] <= cutoff_date]

<h2>Checkpoint - Final</h2>

In [30]:
# Calculate the number of rows
num_rows = len(merged_df)

merged_df['INSPECTION_DATE'] = pd.to_datetime(merged_df['INSPECTION_DATE'])

# Find the oldest and newest dates
oldest_date = merged_df['INSPECTION_DATE'].min()
newest_date = merged_df['INSPECTION_DATE'].max()

# Create the message
message9 = f'contains {num_rows} rows. Oldest Date: {oldest_date}. Newest Date: {newest_date}'
print(message9)

contains 12745 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-06-30 00:00:00


In [31]:
print(message1)
print(message2)
print(message3)
print(message4)
print(message5)
print(message6)
print(message7)
print(message8)
print(message9)

contains 1308301 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 1308301 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 1246391 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 1202805 rows. Oldest Date: 2000-02-10 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 13415 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 12985 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-09-30 00:00:00
contains 12745 rows. Oldest Date: 2007-12-24 00:00:00. Newest Date: 2023-06-30 00:00:00


In [32]:
#data_new = df1

In [33]:
#data_old = pd.read_excel(r"C:\Users\Elisha.Zissman\Social Investment Business\Team_Learning&Influence - Documents\Data Projects\EPC - ez - oh\Community EPCs project\epc_community_buildings.xlsx")

In [34]:
# Identify the new rows based on a specific column
#unique_column = 'BUILDING_REFERENCE_NUMBER'  # Replace with the actual column name you want to use

# Extract the values of the unique column from both DataFrames
#old_values = set(data_old[unique_column])
#new_values = set(data_new[unique_column])

# Find the values that are in new_values but not in old_values
#new_unique_values = new_values - old_values

# Create a DataFrame containing the new rows based on the unique column
#new_rows = data_new[data_new[unique_column].isin(new_unique_values)]

In [35]:
#new_rows_BRN = new_rows

In [36]:
# Identify the new rows based on a specific column
#unique_column = 'LMK_KEY'  # Replace with the actual column name you want to use

# Extract the values of the unique column from both DataFrames
#old_values = set(data_old[unique_column])
#new_values = set(data_new[unique_column])

# Find the values that are in new_values but not in old_values
#new_unique_values = new_values - old_values

# Create a DataFrame containing the new rows based on the unique column
#new_rows = data_new[data_new[unique_column].isin(new_unique_values)]

In [37]:
#data_old = data_old.rename(columns={'LMK_KEY_x':'LMK_KEY'})

In [38]:
#new_rows_LMK = new_rows

In [39]:
#new_rows_LMK.head()

In [40]:
#list(df)

In [41]:
#df = new_rows_LMK

# Assuming you have a DataFrame named 'df' and the column name is 'country code'
#df = df[~df['rgn'].str.startswith(('W', 'S'))]

In [42]:
#df

In [43]:
#df = new_rows_BRN

# Assuming you have a DataFrame named 'df' and the column name is 'country code'
#df = df[~df['rgn'].str.startswith(('W', 'S'))]

In [44]:
#df